In [9]:
import pandas as pd
from surprise import Dataset, SVD, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse,
    mae,
    rsquared,
    exp_var,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    get_top_k_items,
)
from recommenders.models.surprise.surprise_utils import (
    predict,
    compute_ranking_predictions,
)
from recommenders.utils.notebook_utils import store_metadata

In [10]:
file_path = r'C:\Users\rynoc\Python_Analysis\ml-1m.inter'
df = pd.read_csv(file_path, sep='\t')

df.rename(columns={'user_id:token':'userID', 'item_id:token':'itemID', 'rating:float':'rating'}, inplace=True)

df_clean = df.drop('timestamp:float', axis=1) 

df_clean['userID'] = df_clean['userID'].astype(str)
df_clean['itemID'] = df_clean['itemID'].astype(str)

print(df_clean.head())
print(df_clean.info())

  userID itemID  rating
0      1   1193       5
1      1    661       3
2      1    914       3
3      1   3408       4
4      1   2355       5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   userID  1000209 non-null  object
 1   itemID  1000209 non-null  object
 2   rating  1000209 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 22.9+ MB
None


In [6]:

# Initialize the Surprise Reader with the appropriate rating scale (1 to 5 for MovieLens)
reader = Reader(rating_scale=(1, 5))

# Load the DataFrame into Surprise Dataset
data = Dataset.load_from_df(df_clean, reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD model
model = SVD(biased=False)

# Train the model on the training data
model.fit(trainset)

# Test the model on the test data
predictions = model.test(testset)

# Evaluate the model using RMSE
accuracy.rmse(predictions)

# Display sample predictions
for pred in predictions[:5]:
    print(f"User: {pred.uid}, Item: {pred.iid}, Actual: {pred.r_ui}, Predicted: {pred.est:.2f}")

RMSE: 0.8779
User: 3370, Item: 671, Actual: 4.0, Predicted: 3.68
User: 3971, Item: 3147, Actual: 5.0, Predicted: 3.84
User: 1398, Item: 906, Actual: 4.0, Predicted: 3.84
User: 5468, Item: 1945, Actual: 4.0, Predicted: 4.55
User: 3163, Item: 903, Actual: 4.0, Predicted: 4.98


In [4]:
RMSE = accuracy.rmse(predictions)
MSE = accuracy.mse(predictions)
MAE = accuracy.mae(predictions)
fcp = accuracy.fcp(predictions)

RMSE: 0.8734
MSE: 0.7629
MAE:  0.6874
FCP:  0.7498


In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=4.0):
    """
    Calculate precision and recall at k.

    Parameters:
    - predictions: List of predictions from the model.
    - k: Number of top recommendations to consider for precision/recall.
    - threshold: Minimum rating to consider an item as relevant.

    Returns:
    - precision: Precision score.
    - recall: Recall score.
    """
    # Mapping predictions to users
    user_est_true = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))

    precisions = []
    recalls = []

    for uid, user_ratings in user_est_true.items():
        # Sort by estimated rating and take top k
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]

        # Calculate number of relevant items in top k
        relevant_and_recommended = sum((true_r >= threshold) for (_, true_r) in top_k)
        relevant_items = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        precisions.append(relevant_and_recommended / k if k > 0 else 0)
        recalls.append(relevant_and_recommended / relevant_items if relevant_items > 0 else 0)

    # Average precision and recall across all users
    precision = sum(precisions) / len(precisions)
    recall = sum(recalls) / len(recalls)

    return precision, recall

precision, recall = precision_recall_at_k(predictions, k=5, threshold=4.0)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


# Test zone

In [8]:
with Timer() as test_time:
    all_predictions = compute_ranking_predictions(
        model, train, usercol="userID", itemcol="itemID", remove_seen=True
    )

print(f"Took {test_time.interval} seconds for prediction.")

NameError: name 'train' is not defined